In [1]:
from preprocessing import *
from models import *
from train_eval import *

In [2]:

data_name = 'ml_1m_stratified'

ratio = 0
rating_map = None
post_rating_map = None
adj_dropout = 0
max_nodes_per_hop = 100
data_appendix ="_mnph10"
epochs = 40
val_test_appendix = "testmode"

datasplit_path = (
            'raw_data/' + data_name + '/split_seed' + str(1234) + 
            '.pickle'
        )

if data_name in ['flixster', 'douban', 'yahoo_music']:
    (
        u_features, v_features, adj_train, train_labels, train_u_indices, train_v_indices,
        val_labels, val_u_indices, val_v_indices, test_labels, test_u_indices, 
        test_v_indices, class_values
    ) = load_data_monti(data_name, True, rating_map, post_rating_map)
elif data_name == 'ml_100k':
    print("Using official MovieLens split u1.base/u1.test with 20% validation...")
    (
        u_features, v_features, adj_train, train_labels, train_u_indices, train_v_indices,
        val_labels, val_u_indices, val_v_indices, test_labels, test_u_indices, 
        test_v_indices, class_values
    ) = load_official_trainvaltest_split(
        data_name, True, rating_map, post_rating_map, 1.0
    )
elif data_name == "ml_1m_stratified" or  data_name == "goodreads_stratified":
    print("loading from file without features")
    (
        u_features, v_features, adj_train, train_labels, train_u_indices, train_v_indices,
        val_labels, val_u_indices, val_v_indices, test_labels, test_u_indices, 
        test_v_indices, class_values,  testtopk_u_indices, testtopk_v_indices
    ) = load_from_file(
        data_name, True, rating_map, post_rating_map, 1.0
    )

else:
    (
        u_features, v_features, adj_train, train_labels, train_u_indices, train_v_indices,
        val_labels, val_u_indices, val_v_indices, test_labels, test_u_indices, 
        test_v_indices, class_values
    ) = create_trainvaltest_split(
        data_name, 1234, True, datasplit_path, True, True, rating_map, 
        post_rating_map, 1.0
    )

# print("loading from file without features")

# (
#     u_features, v_features, adj_train, train_labels, train_u_indices, train_v_indices,
#     val_labels, val_u_indices, val_v_indices, test_labels, test_u_indices, 
#     test_v_indices, class_values
# ) = load_from_file(
#     data_name, True, rating_map, post_rating_map, 1.0
# )

u_features, v_features = None, None
n_features = 0


train_indices = (train_u_indices, train_v_indices)

test_indices = (test_u_indices, test_v_indices)
l = len(test_u_indices)
testtopk_indices = (testtopk_u_indices, testtopk_v_indices)

# testtopk_indices = (testtopk_u_indices[:l], testtopk_v_indices[:l])



train_graphs, val_graphs, test_graphs = None, None, None

data_combo = (data_name, data_appendix, val_test_appendix)
file_dir = os.path.dirname(os.path.realpath('__file__'))

res_dir = os.path.join(
    file_dir, 'results/{}{}_{}'.format(
        data_name, '', val_test_appendix
    )
)

# dataset_class = 'MyDynamicDataset' 

dataset_class = 'MyDataset'
print('++++++++++++++++= start herererer')

evaled = eval(dataset_class)
print('done here')
print(evaled)

train_graphs = eval(dataset_class)(
    'data/{}{}/{}/train'.format(*data_combo),
    adj_train, 
    train_indices, 
    train_labels, 
    1, 
    1.0, 
    max_nodes_per_hop, 
    u_features, 
    v_features, 
    class_values, 
    max_num=None,
    parallel=False
)


test_graphs = eval(dataset_class)(
    'data/{}{}/{}/test'.format(*data_combo),
    adj_train, 
    test_indices, 
    test_labels, 
    1,
    1.0,
    max_nodes_per_hop, 
    u_features, 
    v_features, 
    class_values, 
    max_num=None,
    parallel=False
)


# test_graphs_topk = eval(dataset_class)(
#     'data/{}{}/{}/testtopk'.format(*data_combo),
#     adj_train, 
#     testtopk_indices, 
#     # test_indices,
#     # test_labels, 
#     np.random.randint(5, size=len(testtopk_indices[0])), 
#     1,
#     1.0,
#     max_nodes_per_hop, 
#     u_features, 
#     v_features, 
#     class_values, 
#     max_num=None,
#     parallel=False
# )
# print(np.random.randint(5, size=len(testtopk_indices)).shape)
# print(len(testtopk_indices[0]))
# print(len(test_graphs_topk))
# print(len(test_graphs))

# num_relations = len(class_values)
# multiply_by = 1

loading from file without features
++++++++++++++++= start herererer
done here
<class 'util_functions.MyDataset'>


Processing...


Enclosing subgraph extraction begins...
35584150
35584150


  0%|                               | 3659/35584150 [00:02<5:30:46, 1792.82it/s]


IndexError: index 3706 is out of bounds for axis 0 with size 3706

In [4]:
num_relations = len(class_values)
multiply_by = 1
model = IGMC (
        train_graphs, 
        latent_dim=[32, 32, 32, 32], 
        num_relations=num_relations, 
        num_bases=4, 
        regression=True, 
        adj_dropout=adj_dropout, 
        force_undirected=False, 
        side_features=False,
        n_side_features=n_features, 
        multiply_by=multiply_by
    )

MyDataset(88077)

In [11]:
# for param in model.parameters():
#     print(param)

# or
for name, param in model.named_parameters():
    print(name, param.shape)


convs.0.weight torch.Size([4, 4, 32])
convs.0.comp torch.Size([5, 4])
convs.0.root torch.Size([4, 32])
convs.0.bias torch.Size([32])
convs.1.weight torch.Size([4, 32, 32])
convs.1.comp torch.Size([5, 4])
convs.1.root torch.Size([32, 32])
convs.1.bias torch.Size([32])
convs.2.weight torch.Size([4, 32, 32])
convs.2.comp torch.Size([5, 4])
convs.2.root torch.Size([32, 32])
convs.2.bias torch.Size([32])
convs.3.weight torch.Size([4, 32, 32])
convs.3.comp torch.Size([5, 4])
convs.3.root torch.Size([32, 32])
convs.3.bias torch.Size([32])
lin1.weight torch.Size([128, 256])
lin1.bias torch.Size([128])
lin2.weight torch.Size([1, 128])
lin2.bias torch.Size([1])


In [12]:
score = test_once(test_graphs, model, 50, logger=None, evalmethod='rmse')


/opt/homebrew/Caskroom/miniforge/base/envs/torchgeo/lib/python3.10/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Testing begins...


100%|█████████████████████████████████████████| 262/262 [00:05<00:00, 48.80it/s]
/Users/mohandodda/classes/cse/mohanIGMC/IGMC/train_eval.py:397: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df = pd.DataFrame(np.array([test_dataset.links[0], test_dataset.links[1], ys, outs ]).T,  columns=['user', 'item', 'actual', 'predicted'])


ValueError: Shape of passed values is (4, 1), indices imply (4, 4)